# sunrise and sunset calculation from coordinates

This notebook uses the package `suntime` to calcualte sunset times at specific gps positions on specific dates. You need to install the package in your python environment, for instance: 

`pip install suntime`

In [1]:
import datetime
import pytz
import suntime
import pandas as pd
import numpy as np

### Define GPS points
Let's make up a set of random GPS points for the sake of demonstration

In practice, you will probably load this informatino from a table using `pd.read_csv`. Take note of the column names for latitude and longitude, they are used below. 
- the values must be in decimal degrees, with West longitudes and South latitudes reported as negatives

In [2]:
points = pd.DataFrame({
    'point_name':[f"point_{i:02n}" for i in range(5)],
    'latitude':np.random.uniform(39,41,5),
    'longitude':np.random.uniform(-77,-79,5)
})
points

,point_name,latitude,longitude
0,point_00,39.963356,-77.010932
1,point_01,40.546199,-77.101676
2,point_02,40.071716,-77.920263
3,point_03,39.632356,-78.042123
4,point_04,40.541967,-78.877888


### Create a list of the dates for which you want to know the sunset time:

this code generates a list of four consecutive dates starting April 15 2020

In [3]:
first_date = datetime.date(2020,4,15)
number_of_days = 4
datelist = pd.date_range(first_date, periods=number_of_days).tolist()

to use a list of specific dates, you could write them explicitly:

In [4]:
# datelist = [
#     datetime.date(2020,4,15),
#     datetime.date(2020,4,20),
#     datetime.date(2020,6,1),
# ]

### Select a time zone. 

The times output by the code will be given in the timezone that you define here. 

_To see a list of timezones that you can select, print `pytz.all_timezones`_

In [5]:
timezone = pytz.timezone('US/Eastern')

This cell defines a helper function to get the sunset time formatted as HH:MM:SS using the package suntime

In [6]:
def get_sunset(latitude,longitude,date):
    """calculate sunset time """
    sun = suntime.Sun(latitude, longitude)
    today_ss = sun.get_sunset_time(date)
    return today_ss.astimezone(timezone).strftime('%H:%M:%S')

#here's the sunrise version
def get_sunrise(latitude,longitude,date):
    """calculate sunset time """
    sun = suntime.Sun(latitude, longitude)
    today_ss = sun.get_sunrise_time(date)
    return today_ss.astimezone(timezone).strftime('%H:%M:%S')

For each date, this cell will add a column to the `points` dataframe containing the sunset time at each location.

In [7]:
sunset_times = points.copy()
for d in datelist:
    col_name = str(d.date()) #use the YYYY-MM-DD date format for the column name
    sunset_times[col_name]=[
        get_sunset(row['latitude'],row['longitude'],d) for _,row in sunset_times.iterrows()
    ]
sunset_times

,point_name,latitude,longitude,2020-04-15,2020-04-16,2020-04-17,2020-04-18
0,point_00,39.963356,-77.010932,19:47:00,19:48:00,19:49:00,19:50:00
1,point_01,40.546199,-77.101676,19:48:00,19:49:00,19:50:00,19:51:00
2,point_02,40.071716,-77.920263,19:51:00,19:52:00,19:53:00,19:54:00
3,point_03,39.632356,-78.042123,19:51:00,19:52:00,19:53:00,19:54:00
4,point_04,40.541967,-78.877888,19:55:00,19:56:00,19:57:00,19:58:00


Repeat for sunrise

In [8]:
sunrise_times = points.copy()
for d in datelist:
    col_name = str(d.date()) #use the YYYY-MM-DD date format for the column name
    sunrise_times[col_name]=[
        get_sunrise(row['latitude'],row['longitude'],d) for _,row in sunrise_times.iterrows()
    ]
sunrise_times

,point_name,latitude,longitude,2020-04-15,2020-04-16,2020-04-17,2020-04-18
0,point_00,39.963356,-77.010932,06:29:00,06:28:00,06:26:00,06:25:00
1,point_01,40.546199,-77.101676,06:29:00,06:27:00,06:26:00,06:24:00
2,point_02,40.071716,-77.920263,06:33:00,06:31:00,06:30:00,06:28:00
3,point_03,39.632356,-78.042123,06:34:00,06:32:00,06:31:00,06:30:00
4,point_04,40.541967,-78.877888,06:36:00,06:35:00,06:33:00,06:32:00
